In [1]:
import numpy as np
import pandas as pd

from power_grid_model import LoadGenType, ComponentType, DatasetType, ComponentAttributeFilterOptions
from power_grid_model import PowerGridModel, CalculationMethod, CalculationType
from power_grid_model import initialize_array, attribute_dtype

In [2]:
#节点
node = initialize_array(DatasetType.input, ComponentType.node, 33)
node['id'] = np.arange(33)#节点编号
node['u_rated'] = 10.5e3#额定电压

#线路
line = initialize_array(DatasetType.input, ComponentType.line, 40)
line['id'] = np.arange(33,73)#线路编号
df_bus = pd.read_csv('Bus.csv')#读取节点文件
from_node = df_bus.iloc[:, 0]#提取第一列
to_node = df_bus.iloc[:, 1]#提取第二列
#转换为numpy
from_node = from_node.to_numpy()
to_node = to_node.to_numpy()
#赋值
line['from_node'] = from_node#线路起始节点
line['to_node'] = to_node#线路终止节点
r_node = df_bus.iloc[:, 2]#提取第三列电阻
x_node = df_bus.iloc[:, 3]#提取第四列电抗
#转换为numpy
r_node = r_node.to_numpy()
x_node = x_node.to_numpy()
#赋值
line['r1'] = r_node
line['x1'] = x_node
line['c1'] = 0
line['tan1'] = 0
line['from_status'] = 1
line['to_status'] = 1
line['i_n'] = 400

#负载
load = initialize_array(DatasetType.input, ComponentType.sym_load, 33)
load['id'] = np.arange(73, 106)#节点编号
df_load = pd.read_csv('loads.csv')#读取文件
load_node = df_load.iloc[:, 0]#提取第一列
load_p = df_load.iloc[:, 1]#提取第二列
load_q = df_load.iloc[:, 2]#提取第三列
#转换为numpy
load_node = load_node.to_numpy()
load_p = load_p.to_numpy()*1e6
load_q = load_q.to_numpy()*1e6
load['node'] = load_node
load['type'] = LoadGenType.const_power
load['p_specified'] = load_p
load['q_specified'] = load_q
load['status'] = 1

#电源
source = initialize_array(DatasetType.input, ComponentType.source, 1)
source['id'] = 106
source['node'] = 0
source['status'] = 1
source['u_ref'] = 1.2057142857#电压参考值

# all
input_data = {
    ComponentType.node: node,
    ComponentType.line: line,
    ComponentType.sym_load: load,
    ComponentType.source: source,
}



In [3]:
pd.DataFrame(input_data[ComponentType.sym_load])

,id,node,status,type,p_specified,q_specified
0,73,0,1,0,3755000.0,2500000.0
1,74,1,1,0,100000.0,60000.0
2,75,2,1,0,90000.0,40000.0
3,76,3,1,0,120000.0,80000.0
4,77,4,1,0,60000.0,30000.0
5,78,5,1,0,60000.0,20000.0
6,79,6,1,0,200000.0,100000.0
7,80,7,1,0,200000.0,100000.0
8,81,8,1,0,60000.0,20000.0
9,82,9,1,0,60000.0,20000.0


In [4]:
from power_grid_model.validation import assert_valid_input_data

assert_valid_input_data(input_data=input_data, calculation_type=CalculationType.power_flow)

In [5]:
model = PowerGridModel(input_data)

In [6]:
result = model.calculate_power_flow()

In [7]:
print("------node result------")
pd.DataFrame(result[ComponentType.node])

------node result------


,id,energized,u_pu,u,u_angle,p,q
0,0,1,1.205231,12654.921671,-0.000488,3.884011e+06,2.592534e+06
1,1,1,1.201619,12616.996115,-0.000134,-1.000000e+05,-6.000000e+04
2,2,1,1.188146,12475.531614,0.000919,-9.000000e+04,-4.000000e+04
3,3,1,1.183702,12428.873509,0.001122,-1.200000e+05,-8.000000e+04
4,4,1,1.179545,12385.219172,0.001289,-6.000000e+04,-3.000000e+04
5,5,1,1.169754,12282.418959,-0.000325,-6.000000e+04,-2.000000e+04
6,6,1,1.168459,12268.822210,-0.001981,-2.000000e+05,-1.000000e+05
7,7,1,1.167047,12253.997629,-0.002198,-2.000000e+05,-1.000000e+05
8,8,1,1.162695,12208.297815,-0.002392,-6.000000e+04,-2.000000e+04
9,9,1,1.162151,12202.589595,-0.002810,-6.000000e+04,-2.000000e+04


In [9]:
# 导纳矩阵
# 初始化33x33的零矩阵
Y = np.zeros((33, 33), dtype=complex)

# 获取线路数据
line = input_data[ComponentType.line]

# 计算自导纳和互导纳
for i in range(len(line)):
    from_node = int(line[i]['from_node'])
    to_node = int(line[i]['to_node'])
    r = line[i]['r1']
    x = line[i]['x1']
    
    # 计算自导纳
    y_ii = 1 / (r + 1j * x)
    
    # 添加自导纳
    Y[from_node, from_node] += y_ii
    Y[to_node, to_node] += y_ii
    
    # 添加互导纳
    Y[from_node, to_node] -= y_ii
    Y[to_node, from_node] -= y_ii

# 打印导纳矩阵
Y

array([[ 8.60890546 -4.3884876j , -8.60890546 +4.3884876j ,
         0.         +0.j        , ...,  0.         +0.j        ,
         0.         +0.j        ,  0.         +0.j        ],
       [-8.60890546 +4.3884876j , 16.60227118-11.29969373j,
        -1.61058367 +0.82031959j, ...,  0.         +0.j        ,
         0.         +0.j        ,  0.         +0.j        ],
       [ 0.         +0.j        , -1.61058367 +0.82031959j,
         6.80189925 -3.98999342j, ...,  0.         +0.j        ,
         0.         +0.j        ,  0.         +0.j        ],
       ...,
       [ 0.         +0.j        ,  0.         +0.j        ,
         0.         +0.j        , ...,  1.88471913 -2.1046965j ,
        -1.36554429 +1.59159574j,  0.         +0.j        ],
       [ 0.         +0.j        ,  0.         +0.j        ,
         0.         +0.j        , ..., -1.36554429 +1.59159574j,
         2.21003965 -2.91950898j, -0.84449537 +1.32791324j],
       [ 0.         +0.j        ,  0.         +0.j        

In [14]:
V=pd.DataFrame(result[ComponentType.node]['u'])#节点电压矩阵
V_np = V.to_numpy()#转换为numpy
I=np.matmul(Y,V_np)#节点电流矩阵
I#节点电流矩阵复数形式

array([[ 3.26497523e+02-1.66435831e+02j],
       [-2.43523539e+01-2.05169422e+01j],
       [-1.47948315e+01-1.19123999e+01j],
       [-1.02711811e+01+5.22868445e+00j],
       [-1.90318300e+01-1.57625807e+01j],
       [-1.74367406e+01+1.73056446e+01j],
       [ 1.26969383e+01+1.39218720e+01j],
       [-1.53243909e+01+1.09854658e+01j],
       [-6.60967459e+00-5.69943952e-01j],
       [-3.40434655e+00+2.50187623e+00j],
       [-4.05745135e+00+1.34163674e+00j],
       [-1.83837841e+00+8.66418327e+00j],
       [-7.95724219e+00-1.23249206e-01j],
       [-5.94095101e+00+1.04268209e+01j],
       [-6.57767925e+00+1.09093072e+01j],
       [-6.92125161e+00-1.98117492e+00j],
       [-3.26042597e+00+2.61307890e+00j],
       [-3.68946311e+00+8.23500907e+00j],
       [-3.32314658e+00+6.94677798e+00j],
       [-1.45067518e+01-2.01708146e+00j],
       [-4.81429324e+00+5.98485313e+00j],
       [-1.84587537e-01+8.51614625e+00j],
       [-1.21469087e+01-2.30083874e+00j],
       [-2.87169398e+01+2.31846186

In [16]:

# 计算模值
magnitudes = np.abs(I)

# 计算角度（默认以弧度为单位）
angles = np.angle(I)

# 将模值和角度组合成一个新的二维数组
I_abs_angle = np.column_stack((magnitudes, angles))

# 打印结果
I_abs_angle

array([[366.47171581,  -0.47142619],
       [ 31.84308489,  -2.44146686],
       [ 18.99453373,  -2.46370535],
       [ 11.52546317,   2.67072048],
       [ 24.71172802,  -2.44987848],
       [ 24.56675106,   2.35996785],
       [ 18.84226001,   0.83138332],
       [ 18.85516941,   2.51963966],
       [  6.63420185,  -3.05557667],
       [  4.22480296,   2.50782161],
       [  4.2735115 ,   2.82225005],
       [  8.85707102,   1.77987685],
       [  7.95819664,  -3.12610496],
       [ 12.0005622 ,   2.08869568],
       [ 12.73887156,   2.11337624],
       [  7.19922065,  -2.86280224],
       [  4.17834403,   2.46596623],
       [  9.02371943,   1.99200391],
       [  7.70071604,   2.01699261],
       [ 14.64631238,  -3.00343414],
       [  7.68087797,   2.24822229],
       [  8.51814648,   1.59246794],
       [ 12.3628981 ,  -2.95439284],
       [ 36.90784705,   2.4623861 ],
       [ 37.38894376,   2.65240802],
       [  5.28417902,   2.66892998],
       [ 12.35675304,  -2.44834111],
 